In [ ]:
# to activate the venv environtment:

# python -m venv venv
# .\venv\Scripts\activate

# Installing the following libraries:


    # pip install torch torchvision torchaudio pytorch-optimizer matplotlib


